# tube

Transport for London (TfL) has asked you to study the feasibility of replacing the manned ticket offices with automated ticket machines in its underground stations. 
As a pilot study, TfL wants you to focus on the evening rush hour period, from 16:00 to 20:00, at a very small station with 1 employee at the ticket office, which will be replaced with 1 machine.

Using historical data provided to you, you have determined that the time between customers arrivals at the ticket office/machine is exponentially distributed with a mean of 3.2 minutes. 
In addition, you have determined using this data that the service time (in minutes) at the manned ticket office can be modeled with a Gamma random variable with shape parameter 2 and scale parameter 1.3. 
Through its own studies, TfL has determined that the service time (in minutes) at the automated ticket machine can be modeled with a Gamma random variable with shape parameter 1 and scale parameter 2.6.

Call the current system "System 1", and call the proposed system "System 2".

Using 100 replications, compute a point estimate of the average delay in System 1 and System 2. Use the t-test for equal means to compare the mean average delay of the two systems. <span style="color:#a00000">Use separate streams of random numbers to generate interarrival times and service times.</span>

In [1]:
##### Setup #####
# Import all simulation functions from SimPy
from SimPy.Simulation import *

# Import mean, std, sqrt functions from numpy
from numpy import mean, std, sqrt

# Import seed initializer and random sampling functions from NumPy
from numpy.random import RandomState, seed, exponential, gamma

# Import Levene test and t-test from scipy.stats
from scipy.stats import levene, ttest_ind

In [2]:
##### Parameters #####
class P:
    # Customers arrive at the ticket office/machine with 
    # exponentially distributed interarrival times with mean 3.2
    interarrivalTimeMean = 3.2
    
    # Service times follow gamma distribution with some shape/scale parameters
    serviceTimeShape = None
    serviceTimeScale = None
    
    # One server: we assume there is 1 employee or 1 machine
    nServers = 1
    
    # Simulation time: 4 hours
    simulationTimeMax =  4 * 60
    
    
##### Streams #####
class S:
    inter = RandomState()
    serv = RandomState()
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        # Record customer arrival time
        arrivalTime = now()
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        # Observe customer delay using monitor
        M.delay.observe(now() - arrivalTime)
        serviceTime = S.serv.gamma(shape = P.serviceTimeShape, scale = P.serviceTimeScale)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = S.inter.exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None
    
    
##### Monitors #####
class M:
    # Delay
    delay = None


##### Model #####
def model(interSeed, servSeed):
    # Initialize SimPy 
    initialize()

    # Initialize seed for interarrival time random number generator
    S.inter.seed(interSeed)

    # Initialize seed for service time random number generator
    S.serv.seed(servSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)
    
    # Create the monitor for customer delay
    M.delay = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    # Compute performance measure: average delay
    avgDelay = M.delay.mean()
    
    # Return performance measure
    return avgDelay

In [3]:
##### Experiment - System 1 #####
# Number of replications
n = 100

# Service time parameters for System 1
P.serviceTimeShape = 2
P.serviceTimeScale = 1.3

# Perform replications, and put performance measure from each replication in one list
avgDelay1Obs = [model(interSeed = 123*i, servSeed = 123*i) for i in range(n)]

# Sample mean to estimate mean performance measure
avgDelay1SM = mean(avgDelay1Obs)
print("SYSTEM 1")
print("Point estimate for mean average delay: {0}".format(avgDelay1SM))

SYSTEM 1
Point estimate for mean average delay: 5.654543994484325


In [4]:
##### Experiment - System 2 #####
# Number of replications
n = 100

# Service time parameters for System 2
P.serviceTimeShape = 1
P.serviceTimeScale = 2.6

# Perform replications, and put performance measure from each replication in one list
avgDelay2Obs = [model(interSeed = 123*i, servSeed = 123*i) for i in range(n)]

# Sample mean to estimate mean performance measure
avgDelay2SM = mean(avgDelay2Obs)
print("SYSTEM 2")
print("Point estimate for mean average delay: {0}".format(avgDelay2SM))

SYSTEM 2
Point estimate for mean average delay: 3.5526531164118755


In [5]:
(LeveneTS, LevenePValue) = levene(avgDelay1Obs, avgDelay2Obs)
print("Levene test p-value = {0}".format(LevenePValue))

Levene test p-value = 2.7266447572960463e-10


Assuming a significance level of 0.05, we reject the null hypothesis of the Levene test, and statistically conclude that the two sets of observations have unequal variances.

In [6]:
(tTS, tPValue) = ttest_ind(avgDelay1Obs, avgDelay2Obs, equal_var = False)
print("t-test p-value = {0}".format(tPValue))

t-test p-value = 3.936707784482137e-09


Assuming a significance level of 0.05, we reject the null hypothesis of the t-test, and statistically conclude that the mean average delay in the two systems are in fact different.